In [1]:
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta
import pymongo
from pymongo import MongoClient
import praw
import pprint

In [2]:
reddit = praw.Reddit(client_id='TADWcvi74CnuJg',
                     client_secret='BvPg9JgtZlFHO3L0x5mQdsBGU-0',user_agent='depreshow' 
#                    ,password=reddit_pass, username=reddit_user   ---ADICIONE A LINHA PARA SAIR DO MODO READONLY
                    )
print(reddit.read_only)

True


## Database Connection


making the proper connections on the database dividing the 3 collections: <br>
    -posts from icb (db_icb_posts)<br>
    -posts from icc (db_icc_posts)<br>
    -comments from icc (db_icc_comments)<br>
for security reasons we dont make any modification on bd icb

In [3]:
db_client = MongoClient('localhost', 27017)
db_icb = db_client.icb
db_icb_posts = db_icb.posts
db_icc = db_client.icc
db_icc_posts = db_icc.posts
db_icc_comments = db_icc.comments

Below we're making the transference of posts from icb to icc, and creating a new field called 'dc', which marks if we have already done the ´Download of Comments' from that post

In [625]:
BATCH_SIZE = 150
cursor = db_icb_posts.find({}).batch_size(BATCH_SIZE)
counter = 0
while cursor.alive:
    batch = []
    while len(batch)<BATCH_SIZE and cursor.alive:
        tmp = next(cursor)  
        tmp['dc'] = 1 if tmp['num_comments'] == 0 else 0
        batch += [tmp]
    counter +=150
    db_icc_posts.insert_many(batch)
    if(counter%1500 == 0):
        print(counter)


BulkWriteError: batch op errors occurred

In [5]:
# Check the number of documents in each collection
# Number must be equals
db_icb_posts.count_documents({}), db_icc_posts.count_documents({})

(778822, 778822)

In [9]:
db_icc_comments.count_documents({}), db_icc_posts.count_documents({'dc':0})

(1573858, 2)

# Comments Downloading

In [28]:
%%time
print("Faltam " + str(db_icc_posts.count_documents({'dc':0})) + " documentos para baixar")
cursor = db_icc_posts.find({'dc':0}, no_cursor_timeout=True).max_await_time_ms(3000000).max_time_ms(None) # 531225
debug = []
counter_insercoes = 0
counter = 0
BATCH_SIZE = 100
suposto_counter = 0
while cursor.alive and cursor.count() > 0:
    try:
        while cursor.alive:
            posts = []
            while cursor.alive and len(posts)<BATCH_SIZE:
                posts += [next(cursor)['_id']]
            posts = reddit.info(posts)
            lista_de_atributos = ['name', 'parent_id', 'link_id','author_fullname', 'body', 'created_utc', 'edited', 'score', 'downs', 'ups']
            print("Carregou os posts")
            for post in posts:
                debug1 =[]
                debug1 += [post]
                comentarios_ainda_nao_buscados = post.comments.replace_more(limit=0)
                comentarios_prontos_para_salvar = vars(post.comments)['_comments']
                ids_dos_comentarios_ainda_nao_buscados = []
                for comentario in comentarios_ainda_nao_buscados:
                    for id in vars(comentario)['children']:
                        ids_dos_comentarios_ainda_nao_buscados += ["t1_" + id]
        #       TALVEZ SEJA POSSIVEL OTIMIZAR COM UM IF
                for busca in reddit.info(ids_dos_comentarios_ainda_nao_buscados):
                    comentarios_prontos_para_salvar += [busca]
                insert_batch =[]
                ids_dos_comentarios_para_salvar =[]
                for a in comentarios_prontos_para_salvar:
                    a = vars(a)
                    new = dict()
                    for x in lista_de_atributos:
                        new[x] = a[x] if x in a else None
                    new['_id'] = new.pop('name')
                    ids_dos_comentarios_para_salvar += [new['_id']]
                    insert_batch +=[new]
                try:
                    debug1+=[insert_batch]
                    debug1+=[ids_dos_comentarios_para_salvar]
                    debug+=[[debug1]]
                    if len(insert_batch) > 0:
                        db_icc_comments.insert_many(insert_batch)
                except Exception as e:
                    print(e)
                else:
                    counter += 1
                    db_icc_posts.update_one({'_id':post.name},{'$set':{'dc':1, 'comments_id':ids_dos_comentarios_para_salvar}})
                if(counter%100==0):
                    print(counter)
    except Exception as e:
        print(e)
        cursor = db_icc_posts.find({'dc':0}, no_cursor_timeout=True).max_await_time_ms(3000000).max_time_ms(None) # 531225

Faltam 0 documentos para baixar


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


CPU times: user 44 µs, sys: 3.08 ms, total: 3.12 ms
Wall time: 735 ms


In [14]:
posts_faltantes = []
for i in db_icc_posts.find({'dc':0}):
    posts_faltantes +=

{'_id': 't3_69ow9k', 'author_fullname': None, 'title': 'Do you hate someone you used to love?', 'selftext': '[removed]', 'created_utc': 1494121904.0, 'edited': 1534042918.0, 'is_self': True, 'locked': False, 'num_comments': 1, 'over_18': False, 'score': 3, 'upvote_ratio': None, 'downs': 0, 'ups': 3, 'dc': 0}
{'_id': 't3_2xn11w', 'author_fullname': 't2_h5uo2', 'title': '15 and fucked up myself', 'selftext': "I want to become an architect when I grow up but parents think I shouldn't because they claim there is no money in the work and it's impossible to find a job in it. I am also very interested in entrepreneuring but I don't want to become one and my parents do. I have a bearded dragon and I've abused him by not feeding him worms as much as she should be, mostly I feed her lettuce and other greens but I know she can't just live off that. I have average grades, A's and B's and an E in Spanish II because my teacher hasn't put in many grades and I have to make up an assainment. I am on th